### Collecting output data

In [1]:
import pandas as pd
import numpy as np
tests = [1,2,3,4,5,6,7,8,9,10,11,12,13]
results_dir = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/11. Results"

### Collecting ME simulation results

In [2]:
# Preparing item id's 
cols = []
for test_no in tests:
    cols.append(f"Test {test_no}")
item_ids = pd.DataFrame(columns = cols)
item_ids["Scenarios"] = ["Scenario 1", "Scenario 2", "Scenario 3", "Scenario 4", "Scenario 5"]
item_ids.set_index("Scenarios", inplace = True)

# Preparing item's fill rates
installation_names = ["Bartlett","Bloemfontein","Capetown","Durban","Gaborone","George",
"Kimberley","Kitwe","Lichtenburg","Middelburg","Nelspruit","Port Elizabeth",
"Richards Bay","Rustenburg","Steelpoort","Windhoek","Wolmaransstad","Johannesburg"]
fillrates_ME_sim_df = pd.DataFrame()
fillrates_ME_sim_df["Dealer names"] = installation_names
fillrates_ME_sim_df.set_index("Dealer names", drop = True, inplace = True)

# Preparing soh
stock_ME_sim_df = pd.DataFrame()
stock_ME_sim_df["Dealer names"] = installation_names
stock_ME_sim_df.set_index("Dealer names", drop = True, inplace = True)

# Preparing backorders
backorders_ME_sim_df = pd.DataFrame()
backorders_ME_sim_df["Dealer names"] = installation_names
backorders_ME_sim_df.set_index("Dealer names", drop = True, inplace = True)

# Preparing cost and inventory data frame
cost_inventory_ME_sim_df = pd.DataFrame(columns = ["Expected system cost", "Expected system stock on hand", "Expected system backorders"])



for test_no in tests:
    path_excel_sim_input_ME = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/10. Testing/Simulation/Input Multi-echelon/InputToSimulation_Test_{test_no}.xlsx"
    path_excel_sim_output_ME = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/10. Testing/Simulation/Output Multi Echelon/OutputFromSimulation_Test_{test_no}_ME.xlsx"
    
    ## Extracting item list:
    item_list_df = pd.read_excel(path_excel_sim_input_ME,usecols = "B")
    item_list_df = item_list_df[9:15]
    item_list_df.columns = ["ID"]
    id_list = []
    for val in item_list_df["ID"]:
        id = val[12:].strip()
        id_list.append(id)
    item_list_df["ID"] = id_list
    item_list_df.reset_index(drop = True, inplace = True)
    
    item_ids[f"Test {test_no}"] = id_list

    # Then fillrates
    cols_mean_fillrate = 'B,E,H,K,N,Q,T,W,Z,AC,AF,AI,AL,AO,AR,AU'
    sim_fillrates_df = pd.read_excel(path_excel_sim_output_ME, usecols = cols_mean_fillrate)
    sim_fillrates_df = sim_fillrates_df[12:17] # Extracting fillrates
    sim_fillrates_df = sim_fillrates_df.transpose()
    sim_fillrates_df.columns = item_list_df["ID"]
    sim_fillrates_df.reset_index(drop = True, inplace = True)

        
    # Adding to the larger dataframe
    for i, item_id in enumerate(item_list_df["ID"]):
        scenario = i+1
        # Extracting warehouse names
        names_df = pd.read_excel(path_excel_sim_input_ME,sheet_name = f"InputScenario{scenario}",
                usecols="K")[7:22].reset_index(drop = True)
        names_df.columns = ["Dealer names"]
        
        scenario_fillrates_df = pd.DataFrame(columns = {"Dealer","Fillrate"})
        scenario_fillrates_df["Dealer"] = names_df
        scenario_fillrates_df["Fillrate"] = sim_fillrates_df[item_id]
        scenario_fillrates_df.set_index("Dealer",drop = True,inplace = True)
        names_df.dropna(inplace = True)

        
    
        for name in names_df["Dealer names"]:
            fillrates_ME_sim_df.loc[name,item_id] = scenario_fillrates_df.loc[name,"Fillrate"]
        # Lastly adding johannesburg
        fillrates_ME_sim_df.loc["Johannesburg",item_id] = sim_fillrates_df[item_id].iloc[15]

    # Stock on hand and backorders
    cols_stock = 'H,I,J,K,L'
    sim_stock_df = pd.read_excel(path_excel_sim_output_ME, usecols = cols_stock)
    sim_stock_df = sim_stock_df[20:36]
    sim_stock_df.columns = item_list_df["ID"]
    sim_stock_df.reset_index(drop = True, inplace = True)
    
    cols_backorders = 'B,C,D,E,F'
    sim_backorders_df = pd.read_excel(path_excel_sim_output_ME, usecols = cols_backorders)
    sim_backorders_df = sim_backorders_df[20:36]
    sim_backorders_df.columns = item_list_df["ID"]
    sim_backorders_df.reset_index(drop = True, inplace = True)
    
    for i, item_id in enumerate(item_list_df["ID"]):
        scenario = i+1
        # Extracting warehouse names
        names_df = pd.read_excel(path_excel_sim_input_ME,sheet_name = f"InputScenario{scenario}",
                usecols="K")[7:22].reset_index(drop = True)
        names_df.columns = ["Dealer names"]
        
        # Stock
        scenario_stock_df = pd.DataFrame(columns = {"Dealer","SoH"})
        scenario_stock_df["Dealer"] = names_df
        scenario_stock_df["SoH"] = sim_stock_df[item_id]
        scenario_stock_df.set_index("Dealer",drop = True,inplace = True)
        names_df.dropna(inplace = True)

        for name in names_df["Dealer names"]:
            stock_ME_sim_df.loc[name,item_id] = scenario_stock_df.loc[name,"SoH"]
        stock_ME_sim_df.loc["Johannesburg",item_id] = sim_stock_df[item_id].iloc[15]

        # Backorders
        scenario_backorders_df = pd.DataFrame(columns = {"Dealer","SoH"})
        scenario_backorders_df["Dealer"] = names_df
        scenario_backorders_df["SoH"] = sim_backorders_df[item_id]
        scenario_backorders_df.set_index("Dealer",drop = True,inplace = True)
        names_df.dropna(inplace = True)

        for name in names_df["Dealer names"]:
            backorders_ME_sim_df.loc[name,item_id] = scenario_backorders_df.loc[name,"SoH"]
        backorders_ME_sim_df.loc["Johannesburg",item_id] = sim_backorders_df[item_id].iloc[15]


    # Lastly other data
    cols_data = "C,E,G"
    sim_data_df = pd.read_excel(path_excel_sim_output_ME,usecols = cols_data)
    sim_data_df.columns = ["Expected system cost", "Expected system stock on hand", "Expected system backorders"]
    sim_data_df = sim_data_df[3:8].reset_index(drop = True)
    sim_data_df["ID"] = item_list_df["ID"]
    sim_data_df.set_index("ID", drop = True, inplace = True)
    cost_inventory_ME_sim_df = pd.concat((cost_inventory_ME_sim_df,sim_data_df))

/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the

### Collecting SE simulation results

In [3]:

# Preparing item's fill rates
fillrates_SE_sim_df = pd.DataFrame()
fillrates_SE_sim_df["Dealer names"] = installation_names
fillrates_SE_sim_df.set_index("Dealer names", drop = True, inplace = True)

# Preparing cost and inventory data frame
cost_inventory_SE_sim_df = pd.DataFrame(columns = ["Expected system cost", "Expected system stock on hand", "Expected system backorders"])

# Preparing soh
stock_SE_sim_df = pd.DataFrame()
stock_SE_sim_df["Dealer names"] = installation_names
stock_SE_sim_df.set_index("Dealer names", drop = True, inplace = True)

# Preparing backorders
backorders_SE_sim_df = pd.DataFrame()
backorders_SE_sim_df["Dealer names"] = installation_names
backorders_SE_sim_df.set_index("Dealer names", drop = True, inplace = True)

for test_no in tests:
    path_excel_sim_input_SE = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/10. Testing/Simulation/Input Single-echelon/InputToSimulation_Test_{test_no}_SE.xlsx"
    path_excel_sim_output_SE = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/10. Testing/Simulation/Output Single Echelon/OutputFromSimulation_Test_{test_no}_SE.xlsx"
    
    ## Extracting item list:
    item_list_df = pd.read_excel(path_excel_sim_input_SE,usecols = "B")
    item_list_df = item_list_df[9:15]
    item_list_df.columns = ["ID"]
    id_list = []
    for val in item_list_df["ID"]:
        id = val[12:].strip()
        id_list.append(id)
    item_list_df["ID"] = id_list
    item_list_df.reset_index(drop = True, inplace = True)
    
    item_ids[f"Test {test_no}"] = id_list

    # Then fillrates
    cols_mean_fillrate = 'B,E,H,K,N,Q,T,W,Z,AC,AF,AI,AL,AO,AR,AU'
    sim_fillrates_df = pd.read_excel(path_excel_sim_output_SE, usecols = cols_mean_fillrate)
    sim_fillrates_df = sim_fillrates_df[12:17] # Extracting fillrates
    sim_fillrates_df = sim_fillrates_df.transpose()
    sim_fillrates_df.columns = item_list_df["ID"]
    sim_fillrates_df.reset_index(drop = True, inplace = True)

    # Adding to the larger dataframe
    for i, item_id in enumerate(item_list_df["ID"]):
        scenario = i+1
        # Extracting warehouse names
        names_df = pd.read_excel(path_excel_sim_input_SE,sheet_name = f"InputScenario{scenario}",
                usecols="K")[7:22].reset_index(drop = True)
        names_df.columns = ["Dealer names"]
        
        scenario_fillrates_df = pd.DataFrame(columns = {"Dealer","Fillrate"})
        scenario_fillrates_df["Dealer"] = names_df
        scenario_fillrates_df["Fillrate"] = sim_fillrates_df[item_id]
        scenario_fillrates_df.set_index("Dealer",drop = True,inplace = True)
        names_df.dropna(inplace = True)

    
        for name in names_df["Dealer names"]:
            fillrates_SE_sim_df.loc[name,item_id] = scenario_fillrates_df.loc[name,"Fillrate"]
        # Lastly adding johannesburg
        fillrates_SE_sim_df.loc["Johannesburg",item_id] = sim_fillrates_df[item_id].iloc[15]


    # Stock on hand and backorders
    cols_stock = 'H,I,J,K,L'
    sim_stock_df = pd.read_excel(path_excel_sim_output_SE, usecols = cols_stock)
    sim_stock_df = sim_stock_df[20:36]
    sim_stock_df.columns = item_list_df["ID"]
    sim_stock_df.reset_index(drop = True, inplace = True)
    
    cols_backorders = 'B,C,D,E,F'
    sim_backorders_df = pd.read_excel(path_excel_sim_output_SE, usecols = cols_backorders)
    sim_backorders_df = sim_backorders_df[20:36]
    sim_backorders_df.columns = item_list_df["ID"]
    sim_backorders_df.reset_index(drop = True, inplace = True)
    
    for i, item_id in enumerate(item_list_df["ID"]):
        scenario = i+1
        # Extracting warehouse names
        names_df = pd.read_excel(path_excel_sim_input_SE,sheet_name = f"InputScenario{scenario}",
                usecols="K")[7:22].reset_index(drop = True)
        names_df.columns = ["Dealer names"]
        
        # Stock
        scenario_stock_df = pd.DataFrame(columns = {"Dealer","SoH"})
        scenario_stock_df["Dealer"] = names_df
        scenario_stock_df["SoH"] = sim_stock_df[item_id]
        scenario_stock_df.set_index("Dealer",drop = True,inplace = True)
        names_df.dropna(inplace = True)

        for name in names_df["Dealer names"]:
            stock_SE_sim_df.loc[name,item_id] = scenario_stock_df.loc[name,"SoH"]
        stock_SE_sim_df.loc["Johannesburg",item_id] = sim_stock_df[item_id].iloc[15]

        # Backorders
        scenario_backorders_df = pd.DataFrame(columns = {"Dealer","SoH"})
        scenario_backorders_df["Dealer"] = names_df
        scenario_backorders_df["SoH"] = sim_backorders_df[item_id]
        scenario_backorders_df.set_index("Dealer",drop = True,inplace = True)
        names_df.dropna(inplace = True)

        for name in names_df["Dealer names"]:
            backorders_SE_sim_df.loc[name,item_id] = scenario_backorders_df.loc[name,"SoH"]
        backorders_SE_sim_df.loc["Johannesburg",item_id] = sim_backorders_df[item_id].iloc[15]


    # Lastly other data
    cols_data = "C,E,G"
    sim_data_df = pd.read_excel(path_excel_sim_output_SE,usecols = cols_data)
    sim_data_df.columns = ["Expected system cost", "Expected system stock on hand", "Expected system backorders"]
    sim_data_df = sim_data_df[3:8].reset_index(drop = True)
    sim_data_df["ID"] = item_list_df["ID"]
    sim_data_df.set_index("ID", drop = True, inplace = True)
    cost_inventory_SE_sim_df = pd.concat((cost_inventory_SE_sim_df,sim_data_df))

/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/opt/anaconda3/envs/master_thesis/lib/python3.10/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the

### Results dataframe
Collecting data from the models and ME and SE simulations to produce a dataframe
containing all relevant values for the analysis of the results.

Will be used to create graphs to visualize results.


In [4]:
# Preparation
# Removing the "REMOVE"
bad_ids = bad_ids = {'REMOVE','REMOVE2','11709051', '11110022', '11110668', 
'11110283', '11143309', '14535304', '13947628','14663703'} 
item_codes_array = item_ids.to_numpy().flatten()
for bad_id in bad_ids:
    item_codes_array = np.delete(item_codes_array, np.where(item_codes_array == bad_id))

In [5]:
column_set = ["Item id", "MMI inventory policy",
    "E[System stock on hand] BM model", "E[System stock on hand] SE model",
    "E[System stock on hand] BM simulation", "E[System stock on hand] SE simulation", 
    "E[System backorders] BM model", "E[System backorders] SE model",
    "E[System backorders] BM simulation", "E[System backorders] SE simulation"]
results_df = pd.DataFrame(columns = column_set)
results_df["Item id"] = item_codes_array
results_df.set_index("Item id", inplace = True)

excel_path = f"{results_dir}/reorder_points_fillrates.xlsx"
writer = pd.ExcelWriter(excel_path)

df_dict = {}
for item_id in item_codes_array:
    # Read Berling-Marklund model output
    path_BM_out = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/outdata_multi_echelon_csv_files/item_{item_id}_output.csv"
    output_BM_model_df = pd.read_csv(path_BM_out)
    output_BM_model_df.set_index("Name", inplace = True, drop = False)
    
    # Read SE model output
    path_SE_out = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/outdata_single_echelon_csv_files/item_{item_id}_output_SE.csv"
    output_SE_model_df = pd.read_csv(path_SE_out)
    output_SE_model_df.set_index("Name", inplace = True, drop = False)

    # Read input files
    path_inputs = f"/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/7. Data collection/item_input_csv_files/item_{item_id}_input.csv"
    input_data_df = pd.read_csv(path_inputs)
    input_data_df.set_index("Name", inplace = True, drop = False)

    # Create a dataframe for the items.
    item_df = pd.DataFrame()
    item_df["Installation name"] = output_BM_model_df["Name"]
    item_df.set_index("Installation name", inplace = True, drop = False)
    item_df.index.name = f"Item: {item_id}"
    
    # Adding demand type
    item_df["Demand type"] = output_BM_model_df["Demand type"]

    # Adding target fill rates IMPORTANT, TAKE FROM SE model
    item_df["Target fill rate"] = output_SE_model_df["Target item fill rate"]

    # Adding Q-values
    item_df["Q"] = output_SE_model_df["Q"]

    # Add R-values and fill rates for BM model and BM simulation
    item_df["R, BM model"] = output_BM_model_df["R optimal"]
    item_df["Fill rate estimate, BM model"] = output_BM_model_df["Realized item fill rate"]
    item_df.loc["Johannesburg","Fill rate estimate, BM model"] = output_BM_model_df.loc["Johannesburg","Target item fill rate"]
    
    item_df["Fill rate estimate, BM simulation"] = -1
    for i,name in enumerate(item_df["Installation name"]):
        item_df.loc[name,"Fill rate estimate, BM simulation"] = fillrates_ME_sim_df.loc[name,item_id]

    # Add R-values and fill rates for SE model and SE simulation
    item_df["R, SE model"] = output_SE_model_df["R optimal"]
    item_df["Fill rate estimate, SE model"] = output_SE_model_df["Realized item fill rate"]
    item_df.loc["Johannesburg","Fill rate estimate, SE model"] = output_SE_model_df.loc["Johannesburg","Realized item fill rate"]
    
    item_df["Fill rate estimate, SE simulation"] = -1
    for i,name in enumerate(item_df["Installation name"]):
        item_df.loc[name,"Fill rate estimate, SE simulation"] = fillrates_SE_sim_df.loc[name,item_id]
    

    # Adding stock on hand 
    item_df["SoH, BM model"] = output_BM_model_df["Expected stock on hand"]
    item_df["SoH, BM simulation"] = -1
    for i,name in enumerate(item_df["Installation name"]):
        item_df.loc[name,"SoH, BM simulation"] = stock_ME_sim_df.loc[name,item_id]
    
    item_df["SoH, SE model"] = output_SE_model_df["Expected stock on hand"]
    item_df["SoH, SE simulation"] = -1
    for i,name in enumerate(item_df["Installation name"]):
        item_df.loc[name,"SoH, SE simulation"] = stock_SE_sim_df.loc[name,item_id]
    
    # Adding backorders
    item_df["Backorders, BM model"] = output_BM_model_df["Expected backorders"]
    item_df["Backorders, BM simulation"] = -1
    for i,name in enumerate(item_df["Installation name"]):
        item_df.loc[name,"Backorders, BM simulation"] = backorders_ME_sim_df.loc[name,item_id]
    
    item_df["Backorders, SE model"] = output_SE_model_df["Expected backorders"]
    item_df["Backorders, SE simulation"] = -1
    for i,name in enumerate(item_df["Installation name"]):
        item_df.loc[name,"Backorders, SE simulation"] = backorders_SE_sim_df.loc[name,item_id]
    
    df_dict[item_id] = item_df

    # Save to csv and excel
    item_df_path = f"{results_dir}/csv_files/{item_id}_results.csv"
    item_df.to_csv(item_df_path)

    excel_sheet = f"Item {item_id}"
    item_df.to_excel(writer,sheet_name = excel_sheet)

    # Add inventory policy setting (Johannesburg policy combined or not)
    results_df.loc[item_id,"MMI inventory policy"] = input_data_df.loc["Johannesburg", "Inventory policy"]
    
    # Add system stock on hand
    results_df.loc[item_id,"E[System stock on hand] BM model"] = output_BM_model_df["Expected stock on hand"].sum()
    results_df.loc[item_id,"E[System stock on hand] SE model"] = output_SE_model_df["Expected stock on hand"].sum()
    results_df.loc[item_id,"E[System stock on hand] BM simulation"] = cost_inventory_ME_sim_df.loc[item_id,"Expected system stock on hand"]
    results_df.loc[item_id,"E[System stock on hand] SE simulation"] = cost_inventory_SE_sim_df.loc[item_id,"Expected system stock on hand"]

    # Add system backorders
    results_df.loc[item_id,"E[System backorders] BM model"] = output_BM_model_df["Expected backorders"].sum()
    results_df.loc[item_id,"E[System backorders] SE model"] = output_SE_model_df["Expected backorders"].sum()
    results_df.loc[item_id,"E[System backorders] BM simulation"] = cost_inventory_ME_sim_df.loc[item_id,"Expected system backorders"]
    results_df.loc[item_id,"E[System backorders] SE simulation"] = cost_inventory_SE_sim_df.loc[item_id,"Expected system backorders"]

    # Add total costs
    results_df.loc[item_id,"E[System cost per time unit] BM model"] = output_BM_model_df["Expected holding costs per time unit"].sum() + output_BM_model_df["Expected shortage costs per time unit"].sum()
    results_df.loc[item_id,"E[System cost per time unit] SE model"] = output_SE_model_df["Expected holding costs per time unit"].sum() + output_SE_model_df["Expected shortage costs per time unit"].sum()
    results_df.loc[item_id,"E[System cost per time unit] BM simulation"] = cost_inventory_ME_sim_df.loc[item_id,"Expected system cost"]
    results_df.loc[item_id,"E[System cost per time unit] SE simulation"] = cost_inventory_SE_sim_df.loc[item_id,"Expected system cost"]



writer.save()

item_ids_path = f"{results_dir}/csv_files/item_ids.csv"
s = pd.Series(item_codes_array)
s.name = "Item id"
s.to_csv(item_ids_path)

results_path = f"{results_dir}/csv_files/results.csv"
results_df.to_csv(results_path)

results_excel_path = f"{results_dir}/stock_backorders_costs.xlsx"
results_df.to_excel(results_excel_path)